# Database Read and Write Examples

## Step 1: Read data from a database table

**The following cell was generated by Insert to Code(pandas data frame) menu. You can either regenerate it for Connected Data defined in your project or modify this code.**

If you are not regenerating code, make sure to verify or change:

1. Connection name
2. Schema name
3. Table name

*Look for "To Do" tag in the code*

In [ ]:
# @hidden_cell

from project_lib import Project
project = Project.access()

# To Do: change or verify connection name. It should match the connection name that you defined in your project
DB2_Cloud_metadata = project.get_connection(name="DB2 Cloud")

import os, jaydebeapi, pandas as pd

DB2_Cloud_url = 'jdbc:db2://{}:{}/{}'.format(
    DB2_Cloud_metadata['host'],
    50000,
    DB2_Cloud_metadata['database']
)

DB2_Cloud_connection = jaydebeapi.connect(
    'com.ibm.db2.jcc.DB2Driver',
    DB2_Cloud_url,
    [DB2_Cloud_metadata['username'],DB2_Cloud_metadata['password']]
)

# To Do: change or schema and table name
query = 'SELECT * FROM "TLR48624"."CUSTOMER_CHURN"'
data_df_1 = pd.read_sql_query(query, con=DB2_Cloud_connection)
data_df_1.head()

# After use, close the database connection with the following code:
# DB2_Cloud_connection.close()


## Step 2: Use the connection object ##
Once we have the connection object, we can run any query. In this cell we retrieve data from a different table. You can try running any SQL query for tables in the same schema. 

In [ ]:
# Once we have the connection object, we can run any query. In this cell we retrieve data from a different table
query = 'SELECT * FROM "TLR48624"."MORTGAGE_DEFAULT"'
mortgageDF = pd.read_sql_query(query, con=DB2_Cloud_connection)
mortgageDF.head()

## Step 3: Insert Data using JayDeBeApi

In [ ]:
# Let's use JayDeBeAPI to insert a row into a database
# Get the cursor object 
curs = DB2_Cloud_connection.cursor()

In [ ]:
# Run the INSERT statement
curs.execute("INSERT INTO CUSTOMER_CHURN VALUES ('F','M',2,100000,'Y',37,2000,5,'CC',2,'K')")

In [ ]:
# Confirm that the row was inserted
curs.execute("SELECT * FROM CUSTOMER_CHURN WHERE CHURN = 'K'")
curs.fetchall()

## Step 4: Insert Data using SQLAlchemy API

The connection string for DB2/SQLAlchemy API has the following format: **db2+ibm_db://userid:password@db_url:db_port/db_name**

Example: *db2+ibm_db://tlr28624:c%40kf12hbpw@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB*

To avoid hardcoding userid and password in the code, you can use the **Insert Credentials** option to reference database userid and password. You can also reference the database name. The example of 

In [ ]:
# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.
Customer_Churn_Table_credentials = project.get_connected_data(name="CustomerChurnTable")

# Get userid
db_username=Customer_Churn_Table_credentials['username']
db_password=Customer_Churn_Table_credentials['password']
db_name=Customer_Churn_Table_credentials['database']

In [ ]:
print(db_username,db_password,db_name)

In [ ]:
# Concatenate the connection string
connection_string = 'db2+ibm_db://' + db_username + ':' + db_password + '@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/' + db_name

In [ ]:
print(connection_string)

In [ ]:
import sqlalchemy

db = sqlalchemy.create_engine(connection_string)
# When we use the SQLAlchemy API, we need to use the engine object
engine = db.connect()
meta = sqlalchemy.MetaData(engine)

In [ ]:
# Insert a row
resultProxy = engine.execute("""INSERT INTO CUSTOMER_CHURN VALUES ('F','M',2,100000,'Y',37,2000,5,'CC',2,'S')""")

In [ ]:
# Verify that the record was added

customerChurnTable = sqlalchemy.Table('CUSTOMER_CHURN', meta, autoload=True, autoload_with=engine)

# In SQLAlchemy this is equivalent to SELECT * FROM CUSTOMER_CHURN WHERE CHURN = 'S'
query = sqlalchemy.select([customerChurnTable]).where(customerChurnTable.columns.churn == 'S')

In [ ]:
res = engine.execute(query)
res.fetchone()

## Step 5: Write a Pandas DataFrame to a Table

In some cases we may want to write an entire data frame to the database. In this example we create a simple pandas data frame and using the combination of SQLAlchemy API and pandas, we can write it to database. 

In [ ]:
# Define required libraries
import numpy as np
import pandas as pd

# Create a dataframe called "random_data" - 4 cols x 100 row 
random_data = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
random_data.head()

In [ ]:
# Write data frame to a new table
# The engine object was created earlier with SQLAlchemy API
random_data.to_sql("PANDAS_TEST1",engine,if_exists='append')

In [ ]:
query = 'SELECT * FROM "TLR48624"."PANDAS_TEST1"'
testDF = pd.read_sql_query(query, con=DB2_Cloud_connection)
testDF.head()

In [ ]:
DB2_Cloud_connection.close()

**Written by Elena Lowery, elowery@us.ibm.com**